In [3]:
import pandas as pd
from collections import Counter
from tqdm import tqdm
import numpy as np

In [4]:
df = pd.read_parquet("data/train_data.pq")
df.head()

,user_id,item_id,date
0,0,0,46
1,0,1,46
2,1,2,46
3,2,3,46
4,3,4,45


In [ ]:
day46 = df[df["date"] == 46]
global_top46 = [item for item, _ in Counter(day46["item_id"]).most_common(1000)]

sample_sub = pd.read_csv("data/sample_submission.csv")
users = sample_sub["user_id"].unique().tolist()

In [7]:
user_items = {}
user_day46_items = {}

In [8]:
for row in tqdm(df.itertuples(index=False), total=len(df)):
    uid, iid, date = row.user_id, row.item_id, row.date

    # Основная история
    if uid not in user_items:
        user_items[uid] = Counter()
    # user_items[uid][iid] += 1
    fresh_weight = np.exp((date - 46) / 7)  # можно подбирать 5–10 в знаменателе
    user_items[uid][iid] += fresh_weight


    # История за 46 день
    if date == 46:
        if uid not in user_day46_items:
            user_day46_items[uid] = set()
        user_day46_items[uid].add(iid)

  0%|          | 0/8777975 [00:00<?, ?it/s]

100%|██████████| 8777975/8777975 [00:44<00:00, 198055.99it/s]


In [9]:
recommendations = {}

for uid in tqdm(users):
    # Топ товаров пользователя по частоте
    base_items = [item for item, _ in user_items.get(uid, Counter()).most_common(5)]

    # Добиваем товарами из 46 дня
    day46_items = list(user_day46_items.get(uid, []))
    for iid in day46_items:
        if iid not in base_items:
            base_items.append(iid)
        if len(base_items) >= 20:
            break

    # Если всё ещё < 20 — добавляем из глобального топа
    for iid in global_top46:
        if len(base_items) >= 20:
            break
        if iid not in base_items:
            base_items.append(iid)

    recommendations[uid] = base_items[:20]

100%|██████████| 293230/293230 [00:05<00:00, 56296.58it/s]


In [10]:
submission = pd.DataFrame({
    "user_id": recommendations.keys(),
    "item_id": [" ".join(map(str, v)) for v in recommendations.values()]
})

submission.to_csv("submission2.csv", index=False)